<a href="https://colab.research.google.com/github/gauthamys/UIC_CS483/blob/main/BigDataMining_HW_1_Gautham.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=953ea0cea29ef477ee128a1a29adecf1bf942b2164e9484df56e56ae3a6245aa
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 39.6 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [5]:
from google.colab import drive, files
drive.mount('/content/drive')

if not os.path.isfile('./soc-LiveJournal1Adj.txt'):
  # upload the file from device
  uploaded = files.upload()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd

# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [7]:
txt = spark.read.text('./soc-LiveJournal1Adj.txt')
txt.show()

+--------------------+
|               value|
+--------------------+
|0\t1,2,3,4,5,6,7,...|
|1\t0,5,20,135,240...|
|2\t0,117,135,1220...|
|3\t0,12,41,55,153...|
|4\t0,8,14,15,18,2...|
|5\t0,1,20,2022,22...|
|6\t0,21,98,2203,3...|
|7\t0,31993,40218,...|
|8\t0,4,38,46,72,8...|
|9\t0,6085,18972,1...|
|10\t0,12,16,30,60...|
|11\t0,1754,6027,7...|
|12\t0,3,10,16,29,...|
|13\t0,12584,32064...|
|14\t0,4,19,19079,...|
|       15\t0,4,27,80|
|16\t0,10,12,18,30...|
|17\t0,19,26,28,95...|
|18\t0,4,16,30,89,...|
|19\t0,14,17,439,1...|
+--------------------+
only showing top 20 rows



In [28]:
def parse_edge(s):
  user, connections = s.split('\t')
  if connections == '':
    connections = []
  else:
    connections = connections.split(',')
  return int(user), list(map(lambda x: int(x), connections))

parse_edge('1\t2,3,4,5')

(1, [2, 3, 4, 5])

In [29]:
def extract(edge):
  user = edge[0]
  connections = edge[1]
  res = []
  for connection in connections:
    key = (user, connection)
    if user > connection:
      key = (connection, user)
    res.append((key, 0))

  for i in range(len(connections)):
    for j in range(i+1, len(connections)):
      key = (connections[i], connections[j])
      if connections[i] > connections[j]:
        key = (connections[j], connections[i])
      res.append((key, 1))

  return res

extract((1, [2,3,4,5]))

[((1, 2), 0),
 ((1, 3), 0),
 ((1, 4), 0),
 ((1, 5), 0),
 ((2, 3), 1),
 ((2, 4), 1),
 ((2, 5), 1),
 ((3, 4), 1),
 ((3, 5), 1),
 ((4, 5), 1)]

In [35]:
rdd = txt.rdd
res = rdd.map(lambda r: r[0]) \
  .map(lambda x: parse_edge(x)) \
  .map(lambda e: extract(e)) \
  .flatMap(lambda x: x) \
  .reduceByKey(lambda x, y: x+y) \
  .map(lambda x: (x[0][0], (x[0][1], x[1])) ) \
  .groupByKey() \
  .mapValues(lambda x: list(x)) \
  .map(lambda x: (x[0], list(sorted(x[1], key=lambda y: (-y[1], y[0])))[:10] )) \
  .collect()

In [44]:
for node, recommendations in res:
    print(f'{node}\t' + ','.join(str(x[0]) for x in recommendations))

Streaming output truncated to the last 5000 lines.
35801	44892,35819,35851,38737,38743,38752,38780,38787,35811,35847
4787	24230,4792,23297,28718,4795,4813,28862,36559,36592,7427
13497	13509,13514,13510,13551,13498,13503,13515,13522,13552,15285
1762	1763,1764,1765,1766,1767,1768,1769,1770,1771,1772
6249	6270,6257,6295,6282,28061,6250,6251,6275,28041,6252
10693	10747,23179,10694,10695,10696,10697,10698,10699,10700,10701
27178	27181,27185,27188,27179,27180,27182,27184,27193,42203,49401
4874	4875,4876,4877,4878,4879,4880,4881,4882,4883,4884
300	22239,1432,6472,6611,6653,37881,1077,1356,1406,1841
23526	23527,23528,23529,23530,23531,23532,23533,23534,23535,23536
22968	24541,30413,22971,24733,35128,24512,24555,22991,22995,23694
5730	5731,5732,28505,31100,18136,20937,22293,24128,24129,28504
30540	30566,30595,30560,30562,30541,30543,30544,30556,30557,30559
17707	17708,17709,17710,17711,17712,17713,17714,18282,18576,18590
6855	28559,6857,6881,6856,17061,28554,28564,28572,29012,6875
17620	17629,2

In [45]:
for node, recommendations in res:
  if node in [8997, 2791, 4985, 8961, 4049, 5060, 739, 1724, 9550, 3151]:
    print(f'{node}\t' + ','.join(str(x[0]) for x in recommendations))

5060	5086,14271,5061,5062,5063,5064,5065,5066,5067,5068
8961	8968,8974,12241,8964,8973,8962,8965,8963,8966,8972
4985	4986,4987,4988,4989,4990,4991,4992,4993,4994,4995
3151	3559,3161,43162,3230,3450,8692,3162,3171,3205,3218
9550	9554,9558,9551,9552,9553,9555,9556,9557,9559,9560
2791	19485,21185,8783,13280,18359,18363,23667,35740,5996,11865
1724	1731,1814,5638,25035,27253,33377,1725,1726,1727,1728
8997	8998,9001,9003,9008,9009,8999,9000,9002,9004,9005
739	21526,28064,762,4169,4795,14474,24208,29610,758,4792
4049	4871,4875,4889,8492,8685,4074,4166,4173,4182,4222
